In [ ]:
import pandas as pd
import sys
sys.path.append('..')
from transformers import KmerTransformer

In [ ]:
df = pd.read_csv("../data/short_viral_host_data.csv")

In [ ]:
from evo2 import Evo2

evo2_model = Evo2('evo2_7b_base')

In [ ]:
import torch

def get_final_token_embedding(sequence, model, layer_name):
    input_ids = torch.tensor(
        model.tokenizer.tokenize(sequence),
        dtype=torch.int,
    ).unsqueeze(0).to('cuda:0')
    with torch.no_grad():
        _, embeddings = model(input_ids, return_embeddings=True, layer_names=[layer_name])
    return embeddings[layer_name][0, -1, :].cpu().to(torch.float32).numpy()

In [ ]:
import numpy as np
from tqdm import tqdm

embedding_list = []
layer_name = 'blocks.26'

for _, row in tqdm(positions.iterrows(), total=len(positions), desc="Extracting embeddings"):
    emb = get_final_token_embedding(row['sequence'], evo2_model, layer_name)
    embedding_list.append(emb)

# Add to DataFrame
df['embedding'] = embedding_list

In [ ]:
df.to_csv("../data/evo2_short_viral_host_data.csv", index=False)